# Bifrost
FrostAura Decision Engine

## Install Dependencies
Python Kernel Version: 3.9.7

In [1]:
!pip install ta-lib

## Import Dependencies

In [24]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import talib
from talib import RSI, BBANDS

## Prepare Data
Load the data produced from [FrostAura Plutus](https://github.com/faGH/fa.services.plutus).

### Data Parsing Utils

In [12]:
def get_pair_name(path: str) -> str:
    pair_name = (path
                    .split('/')[-1]
                    .split('.json')[0]
                    .split('-')[0])
    
    return pair_name

def load_path_data(path: str, frequency: str) -> pd.DataFrame:
    try:
        df = pd.read_json(path)
        df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
        df['ds'] = pd.to_datetime(df['time'], unit='ms')
        df.set_index('ds', inplace=True)
        
        return df.asfreq(freq=frequency)
    except Exception as e:
        print(f'Failed to load {path} with error "{str(e)}". Skipping.')
        return None
    
def load_plutus_pair_data_from_dir(fa_plutus_data_relative_path: str, frequency: str='1H') -> dict:
    file_names = [ f for f in listdir(fa_plutus_data_relative_path) if (isfile(join(fa_plutus_data_relative_path, f)) and '.json' in f and frequency.lower() in f.lower()) ]
    file_paths = np.array([f'{fa_plutus_data_relative_path}/{f}' for f in file_names ])
    market_data = { get_pair_name(path):load_path_data(path, frequency) for path in file_paths if load_path_data(path, frequency) is not None }
    
    return market_data

### Load Raw Pairs Data

In [13]:
fa_plutus_data_relative_path = './../../../fa.services.plutus/user_data/data/binance'
market_data = load_plutus_pair_data_from_dir(fa_plutus_data_relative_path, frequency='4H')

Failed to load ./../../../fa.services.plutus/user_data/data/binance/BOT_BTC-4h.json with error "Length mismatch: Expected axis has 0 elements, new values have 6 elements". Skipping.
Failed to load ./../../../fa.services.plutus/user_data/data/binance/BQX_BTC-4h.json with error "Length mismatch: Expected axis has 0 elements, new values have 6 elements". Skipping.
Failed to load ./../../../fa.services.plutus/user_data/data/binance/NANO_BTC-4h.json with error "Length mismatch: Expected axis has 0 elements, new values have 6 elements". Skipping.


In [22]:
# Visualize the first loaded pair.
market_data[next(iter(market_data))].head(1)

,time,open,high,low,close,volume
ds,,,,,,
2022-04-15,1649980800000,0.000001,0.000001,0.000001,0.000001,60778


### Generate External Regressors
We calculate various market indicator values for all pairs.

#### External Regressors Utils

In [46]:
def featurize_market_data(market_data: dict) -> dict:
    # Perform the transformation for each pair.
    for pair in market_data:
        # Featurizing pair "{pair}" with seasonality.
        pair_df = market_data[pair]
        pair_df['hour'] = pair_df.apply(lambda row: row.name.hour, axis=1)
        pair_df['day'] = pair_df.apply(lambda row: row.name.day, axis=1)
        pair_df['week'] = pair_df.apply(lambda row: row.name.week, axis=1)
        pair_df['month'] = pair_df.apply(lambda row: row.name.month, axis=1)
        
        # Featurizing pair "{pair}" with indicators.
        pair_df['sma'] = talib.SMA(pair_df['close'])
        time_periods = [ 14, 20 ]
        stds = [ 2, 3, 4 ]
        
        for period in time_periods:
            pair_df[f'rsi_tp{period}'] = RSI(pair_df['close'], timeperiod=period)
            pair_df['adx'] = talib.ADX(pair_df['high'], pair_df['low'], pair_df['close'], timeperiod=period)
            slowk, slowd = talib.STOCH(pair_df['high'], pair_df['low'], pair_df['close'], fastk_period=period)
            pair_df[f'stoch_slow_k_tp_{period}'] = slowk
            pair_df[f'stoch_slow_d_tp_{period}'] = slowd
            
            for deviation in stds:
                upper, middle, lower = BBANDS(pair_df['close'], timeperiod=period, nbdevup=deviation, nbdevdn=deviation, matype=0)
                pair_df[f'bb_upper_tp{period}_sd{deviation}'] = upper
                pair_df[f'bb_middle_tp{period}_sd{deviation}'] = middle
                pair_df[f'bb_lower_tp{period}_sd{deviation}'] = lower
                
        # We drop NANs as there will be some leading NANs due to how certain indicators are calculated. For this reason we ignore those records.
    
        market_data[pair] = market_data[pair].dropna()
        
    return market_data

#### Generate Dataframe With External Regressors Included

In [47]:
featurized_market_data = featurize_market_data(market_data)

In [48]:
# Visualize the first loaded pair.
featurized_market_data[next(iter(featurized_market_data))].head(1)

,time,open,high,low,close,volume,hour,day,week,month,...,stoch_slow_d_tp_20,bb_upper_tp20_sd2,bb_middle_tp20_sd2,bb_lower_tp20_sd2,bb_upper_tp20_sd3,bb_middle_tp20_sd3,bb_lower_tp20_sd3,bb_upper_tp20_sd4,bb_middle_tp20_sd4,bb_lower_tp20_sd4
ds,,,,,,,,,,,,,,,,,,,,,
2022-04-21 12:00:00,1650542400000,0.000001,0.000001,0.000001,0.000001,839225,12,21,16,4,...,29.012346,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
